# A script to compare TFRs to fireclusters at different confidence

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
#create geodataframe with files from script´s directory


#paths and filenames
fire_path = r"D:\UNIGIS\MASTER\DownloadedData\WFS\\"
tfr_path=r"D:\UNIGIS\MASTER\Scripts\\"

fire_file = "wfs-area-export_FIRs_Boundary_08-102021_con_pt5.geojson"
# a second GeoJSON containing all fireclusters, also of low confidence, is needed
fire_file_low_conf = "wfs-area-export_2021-08-01-2021-10-31_FIRs_Boundary_08-102021.geojson"

tfr_file = "All_TFRs.geojson"

#read geojson files
gdf_fires_low = gpd.read_file(fire_path + fire_file_low_conf)

gdf_All_TFRs = gpd.read_file(tfr_path + tfr_file)
gdf_fires = gpd.read_file(fire_path + fire_file)

#output filename (if any):
outfile = 'TFRs_without_fires.geojson'

In [3]:
# left join with predicate intersect
gdf_TFR_nofire = gpd.sjoin(gdf_All_TFRs, gdf_fires, how='left', predicate='intersects')

In [4]:
# change the global options that Geopandas inherits from if more shall be displayed
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)

In [5]:
gdf_TFR_nofire = gdf_TFR_nofire[gdf_TFR_nofire["id"].isnull()]

In [6]:
print(len(gdf_TFR_nofire.index))

56


In [7]:
#gdf_TFR_nofire

Sort out eventually erroneous TFRs, cancelled before becoming effective, maybe because they were issued in the wrong place

In [8]:
gdf_TFR_nofire_err = gdf_TFR_nofire[ pd.to_datetime(gdf_TFR_nofire["Cancel_Date__UTC_"]) < pd.to_datetime(gdf_TFR_nofire["Effective_Date__UTC_"]) ]

In [9]:
print(len(gdf_TFR_nofire_err.index))

0


Sort out TFRs issued before fire cluster timeframe (starting 08/01/2021 0000). Their comparison may suffer from a boundary value problem

In [10]:
gdf_TFR_nofire_old = gdf_TFR_nofire[ pd.to_datetime(gdf_TFR_nofire["Issue_Date__UTC_"]) < pd.to_datetime('08/01/2021 0000') ]

In [11]:
print(len(gdf_TFR_nofire_old.index))

17


In [12]:
gdf_TFR_nofire = gdf_TFR_nofire[ pd.to_datetime(gdf_TFR_nofire["Issue_Date__UTC_"]) >= pd.to_datetime('08/01/2021 0000') ]

The remaining TFRs are now joined on fire clusters with a lower confidence

In [13]:
#If it is ever needed to join a join result again, index/ other needed columns need to get renamend
gdf_TFR_nofire.rename(columns = {'index_right':'old_index_right'}, inplace = True)
gdf_TFR_nofire.rename(columns = {'id':'old_id'}, inplace = True)
# left join with predicate intersect
gdf_TFR_nofire_low = gpd.sjoin(gdf_TFR_nofire, gdf_fires_low, how='left', predicate='intersects')

In [14]:
#gdf_TFR_nofire_low

In [15]:
gdf_TFR_nofire_low = gdf_TFR_nofire_low[gdf_TFR_nofire_low["id"].isnull()]

In [16]:
print(len(gdf_TFR_nofire_low.index))

3


In [17]:
gdf_TFR_nofire_low

,OBJECTID,Source,Location,NOTAM__,Issue_Date__UTC_,Effective_Date__UTC_,Cancel_Date__UTC_,Expiration_Date__UTC_,NOTAM_Condition_or_LTA_Subject,Radius,...,index_right,id,age_right,area_right,num_fires_right,confidence_right,newest_detection_right,oldest_detection_right,newest_acquisition_right,oldest_acquisition_right
23,24,ZSE_2021-08-01,ZSE,1/9095,08/01/2021 1555,08/01/2021 1615,08/01/2021 2337,10/01/2021 0500,"!FDC 1/9095 ZSE OR..AIRSPACE 22NM S MEDFORD, O...",3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,541,ZAB_2021-10-20,ZAB,1/5987,10/20/2021 2254,10/20/2021 2245,10/21/2021 0229,10/22/2021 0230,!FDC 1/5987 ZAB AZ..AIRSPACE 23NM SE OF TUCSON...,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
541,542,ZAB_2021-10-21,ZAB,1/6009,10/21/2021 0034,10/21/2021 1400,10/22/2021 2316,11/21/2021 0300,!FDC 1/6009 ZAB AZ..AIRSPACE 23NM SE OF TUCSON...,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# GeoJSON output
if len(gdf_TFR_nofire_low.index)>0:
    gdf_TFR_nofire_low.to_file(filename= outfile, driver='GeoJSON')

D:\anaconda3\envs\master_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
